In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from ucimlrepo import fetch_ucirepo
power_consumption = fetch_ucirepo(id=235)

C:\Users\ronal\anaconda3\envs\UCA_ML\lib\site-packages\ucimlrepo\fetch.py:97: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


In [3]:
X = power_consumption.data.features 
y = power_consumption.data.targets

In [4]:
X.shape

(2075259, 9)

In [6]:
X.head()

Date      Time Global_active_power Global_reactive_power  Voltage  \
0  16/12/2006  17:24:00               4.216                 0.418  234.840   
1  16/12/2006  17:25:00               5.360                 0.436  233.630   
2  16/12/2006  17:26:00               5.374                 0.498  233.290   
3  16/12/2006  17:27:00               5.388                 0.502  233.740   
4  16/12/2006  17:28:00               3.666                 0.528  235.680   

  Global_intensity Sub_metering_1 Sub_metering_2  Sub_metering_3  
0           18.400          0.000          1.000            17.0  
1           23.000          0.000          1.000            16.0  
2           23.000          0.000          2.000            17.0  
3           23.000          0.000          1.000            17.0  
4           15.800          0.000          1.000            17.0

In [7]:
X = X[["Date", "Time", "Global_active_power", "Voltage", "Global_intensity"]]
X.shape

(2075259, 5)

In [8]:
X = X.rename(columns = {"Global_active_power":'P', "Voltage":'V', "Global_intensity":'I'})
X.head()

Date      Time      P        V       I
0  16/12/2006  17:24:00  4.216  234.840  18.400
1  16/12/2006  17:25:00  5.360  233.630  23.000
2  16/12/2006  17:26:00  5.374  233.290  23.000
3  16/12/2006  17:27:00  5.388  233.740  23.000
4  16/12/2006  17:28:00  3.666  235.680  15.800

In [10]:
X["P"] = pd.to_numeric(X["P"], errors="coerce")
X["V"] = pd.to_numeric(X["V"], errors="coerce")
X["I"] = pd.to_numeric(X["I"], errors="coerce")
X.isna().sum()

Date        0
Time        0
P       25979
V       25979
I       25979
dtype: int64

In [11]:
X = X.dropna()
X.shape

(2049280, 5)

In [12]:
X.head()

Date      Time      P       V     I
0  16/12/2006  17:24:00  4.216  234.84  18.4
1  16/12/2006  17:25:00  5.360  233.63  23.0
2  16/12/2006  17:26:00  5.374  233.29  23.0
3  16/12/2006  17:27:00  5.388  233.74  23.0
4  16/12/2006  17:28:00  3.666  235.68  15.8

In [14]:
X['P'].shift(periods = [2, 1, 0, -1]).head()

P_2    P_1    P_0   P_-1
0    NaN    NaN  4.216  5.360
1    NaN  4.216  5.360  5.374
2  4.216  5.360  5.374  5.388
3  5.360  5.374  5.388  3.666
4  5.374  5.388  3.666  3.520

In [15]:
X_P = X['P'].shift(periods = [2, 1, 0, -1]).dropna()
X_V = X['V'].shift(periods = [2, 1, 0]).dropna()
X_I = X['I'].shift(periods = [2, 1, 0]).dropna()

In [16]:
X_preparado = X_P.join(X_V).join(X_I)
X_preparado.head()

P_2    P_1    P_0   P_-1     V_2     V_1     V_0   I_2   I_1   I_0
2  4.216  5.360  5.374  5.388  234.84  233.63  233.29  18.4  23.0  23.0
3  5.360  5.374  5.388  3.666  233.63  233.29  233.74  23.0  23.0  23.0
4  5.374  5.388  3.666  3.520  233.29  233.74  235.68  23.0  23.0  15.8
5  5.388  3.666  3.520  3.702  233.74  235.68  235.02  23.0  15.8  15.0
6  3.666  3.520  3.702  3.700  235.68  235.02  235.09  15.8  15.0  15.8

In [17]:
X_preparado.shape

(2049277, 10)

In [18]:
y_new = X_preparado[["P_-1"]]
X_new = X_preparado.drop("P_-1", axis=1)
X_new.shape, y_new.shape

((2049277, 9), (2049277, 1))

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(X_new, test_size=100000, random_state=42)
X_train.shape, X_test.shape

((1949277, 9), (100000, 9))

In [20]:
y_train, y_test = train_test_split(y_new, test_size=100000, random_state=42)
y_train = np.reshape(y_train, -1)
y_test = np.reshape(y_test, -1)
y_train.shape, y_test.shape

((1949277,), (100000,))

In [21]:
from sklearn.neural_network import MLPRegressor
modelo = MLPRegressor(hidden_layer_sizes=(15, 5), max_iter=1500)

In [22]:
modelo.fit(X_train, y_train)

MLPRegressor(hidden_layer_sizes=(15, 5), max_iter=1500)

In [23]:
y_pred = modelo.predict(X_test)

In [24]:
rmse = np.sqrt(np.mean(np.square(y_test - y_pred)))
nrmse = (rmse / (y_train.max() - y_train.min())) * 100
print("Test NRMSE:", round(nrmse,4), '%')

Test NRMSE: 2.3945 %
